![Pollinator instructions](https://i.imgur.com/6rCfYU2.png)

# Pollinator
Does the heavy-lifting for **Pollinations.AI** on Google Colab's free GPUs.

In [ ]:
#@title Save to Google Drive
#@markdown Will periodically save your results to Google Drive under the folder `/pollinations`

save_to_google_drive = False  #@param {type: "boolean"}

if save_to_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
#@title Config and Requirements

!pip install --upgrade pip
!pip install --upgrade typing-extensions==3.10.0.2 protobuf==3.20.1
!pip install papermill json5
#!pip uninstall -y tqdm
#!pip install git+https://github.com/voodoohop/tqdm.git
from IPython.display import clear_output
from os.path import join, basename
from shlex import quote

from glob import glob
from json5 import loads

from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

clear_output()
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
ipfs_root = "/content/ipfs"
worker_root = "/content"
notebook_path = join(ipfs_root,"input", "notebook.ipynb")

print(f"---Settings---\nIPFS_ROOT: {ipfs_root}")

node_id = None
if os.path.exists("/home/ubuntu/node_id"):
  with open("/home/ubuntu/node_id") as f:
    node_id = f.readline()


In [ ]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output

clear_output()

In [ ]:
#@title Install Pollinations CLI

!wget -N https://nodejs.org/dist/v16.14.0/node-v16.14.0-linux-x64.tar.xz
!tar -xf node-v16.14.0-linux-x64.tar.xz
!rm -rf /tools/node/*
!cp -r /content/node-v16.14.0-linux-x64/* /tools/node


%cd /content
!git clone https://github.com/pollinations/pollinations-ipfs.git
%cd /content/pollinations-ipfs
!git pull
!git checkout main
!git pull
!npm run install_backend
%cd -

clear_output()

In [ ]:
#@title Get GPU and Node ID

#!/content/pollinations/ipfs/ipfsd_install_colab.sh
#!nohup /content/pollinations/ipfs/ipfsd_forever.sh &
#!sleep 5
clear_output()

if node_id is None:
  node_id=!tr -dc A-Za-z0-9 </dev/urandom | head -c 13 ; echo ''
  node_id=node_id[0]

gpu_description=!nvidia-smi -L
gpu_description=gpu_description[0]


print("---Node ID---:", node_id, gpu_description)

In [ ]:
#@title Create connection to frontend UI

import IPython
# from google.colab import output

import urllib.request

def iframe_js(url):
    code = f'''
      console.log("appending iframe of {url}")
      const iframe = document.createElement("iframe")
      iframe.frameBorder = 0
      iframe.width = 250
      iframe.height = 250
      iframe.src = `{url}/incolab.html?node={node_id}&gpu={gpu_description}`
      document.querySelector("#output-area").appendChild(iframe)
      console.log("appended iframe of {url}")
      '''
    display(IPython.display.Javascript(code))

iframe_js("https://pollinations.ai")
iframe_js("https://seedling.pollinations.ai")
#iframe_js("http://localhost:3000")


In [ ]:
#@title Notebook Runner
from time import sleep
import sys
import matplotlib
matplotlib.use('Agg')
!pip config set global.progress_bar off
output_path = ipfs_root+"/output"

!DEBUG=* pollinate-cli.js --execute "run_notebook.sh" -l $output_path/log -p $ipfs_root --ipns -n $node_id --debounce 10000 > /content/cid
